<a href="https://colab.research.google.com/github/bilal-rachik/detection-in-contract-management/blob/master/detectron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handwriting,signatures and initials detection in contract management __Part 3 __ Detectron2.
Dans l'article précédent, nous avons formé notre modèle de détection des écritures manuscrites, signatures , intials  à l'aide de l'API de détection d'objets de TensorFlow.

Dnas cet article nous allons découvrire un autre framework  pour la construction des modèles de détection d'objets et de segmentation d'images, qui s'appelle Detectron2.


Dans cet article, nous allons découvrir un autre cadre pour la construction de modèles de détection d'objets et de segmentation d'images, qui s'appelle Detectron2.

Parallèlement à la sortie de la version 1.3 de PyTorch, Facebook a également publié une réécriture fondamentale de son cadre de détection d'objets Detectron . Le nouveau framework s'appelle Detectron2 et est maintenant implémenté dans PyTorch .

Detectron2 nous permet facilement de construire des modèles de détection d'objets. Cet article vous aidera à démarrer avec Detectron2 en apprenant à  former votre propre modèle.


# Installer Detectron2
L'installation de Detectron2 est facile par rapport à d'autres frameworks de détection d'objets comme l'API de détection d'objets Tensorflow.

# Installer Detectron2
L'installation de Detectron2 est facile par rapport à d'autres frameworks de détection d'objets comme l'API de détection d'objets Tensorflow.

# Installation sur Google Colab
Si vous travaillez dans Google Colab, il peut être installé avec les quatre lignes suivantes:

In [0]:

# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 723.9MB 25kB/s 
     |████████████████████████████████| 4.1MB 28.1MB/s 
  Found existing installation: torch 1.5.0+cu101
    Uninstalling torch-1.5.0+cu101:
      Successfully uninstalled torch-1.5.0+cu101
  Found existing installation: torchvision 0.6.0+cu101
    Uninstalling torchvision-0.6.0+cu101:
      Successfully uninstalled torchvision-0.6.0+cu101
     |████████████████████████████████| 276kB 8.8MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=feaa924bccce9eb96c14a820c2619780e7742a08880f3fbbce969a93f62726fd
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-b

In [0]:

# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html
     |████████████████████████████████| 6.2MB 653kB/s 
  Created wheel for fvcore: filename=fvcore-0.1.dev200506-cp36-none-any.whl size=40569 sha256=b6703c85f697170f8be8827517bceebf4f8203ff26ca848f0ef095e7923015ec
  Stored in directory: /root/.cache/pip/wheels/3f/db/ab/307827ba85b8cc1d288f096909499ecdbcac90db4edaa088fe
Successfully built fvcore


# Installation sur une machine locale
Si vous travaillez sur une machine locale, ce n'est pas aussi simple que cela, mais toujours gérable.

je vous invite à  consulter le guide [d'installation officiel](https://github.com/facebookresearch/detectron2/blob/master/INSTALL.md#common-installation-issues) 



Annoter les images
C'est déja fait et bien expliqué  dans mes [article precedent](https://github.com/bilal-rachik/detection-in-contract-management/blob/master/Handwriting%2Csignatures%20and%20initials%20%20detection%20in%20contract%20management%20%20part%201.ipynb) . 

Nous avons utilisé localement le logiciel open source VOTT développé par Microsoft, pour annoter les contrats.

Notre ensemble de données est composé d'une variété de contrats du gouvernement télécharger depuis ce [lien]( https://www.gsa.gov/real-estate/real-estate-services/leasing-policy-procedures/lease-documents)  .

L'utilisation de VOTT nous a permis de produire un ensemble de formation de 300  images étiquetées à partir d'un échantillon de contrats du gouvernement en quelques heures. Nous avons tiré notre ensemble de tests de 90 images contractuelles supplémentaires.

Une fois que vous avez fini d'annoter votre jeu de données d'image, exportez vos actifs dans VOTT au format Pascal VOC.
Structure d'ensemble de données attendue pour Pascal VOC est la suivante :


In [0]:
pascalVOC-export/
  Annotations/
  ImageSets/
    Main/
      train.txt
      test.txt
      # train.txt or val.txt, if you use these splits
  JPEGImages/

* annotations: Le dossier contient des fichiers XML, un fichier par image. Il stocke des métadonnées sur une image comme un dossier où l'image est stockée, son nom de fichier, sa taille et chaque boîte englobante...etc
* Ensuite, nous avons un JPEGImages dossier, il contient tous nos images jpg.
* train.txt et test.txt : est notre ensemble de données divisé pour former et tester le modèle.

# Enregistrer le jeu de données

Pour que Detectron2 sache comment obtenir l'ensemble de données, nous devons l'enregistrer et éventuellement enregistrer des métadonnées pour votre ensemble de données.
Le processus est bien décrit avec des détails dans la [documentation Detectron2](https://detectron2.readthedocs.io/tutorials/datasets.html) .

En général, Detectron2 utilise son propre format pour la représentation des données qui est similaire aux annotations JSON de COCO. Il s'agit d'implémenter une fonction qui renvoie toutes les informations nécessaires sur les données sous forme de liste et en transmettant le résultat à DatasetCatalog.register



In [0]:
def get_dicts():
  ...
  return list[dict] in the following format

from detectron2.data import DatasetCatalog
DatasetCatalog.register("my_dataset", get_dicts)

Pour l'ensemble de données qui est déjà au format COCO, Detectron2 fournit la `register_coco_instances` fonction qui `load_coco_json` vous enregistrera et ajoutera des métadonnées sur votre ensemble de données.
Les métadonnées sont un mappage de valeurs-clés qui fournit des informations sur l'ensemble de données comme les noms des classes, les couleurs des classes, la racine des fichiers, etc. qui sont accessibles via `MetadataCatalog.get(dataset_name).some_metadata.`

qui pourrait être une inspiration pour nous.


Dans notre cas, nous avons l'ensemble de données au format Pascal VOC et il n'y a pas de chargeur à usage général pour ce format. Heureusement, Detectron2 a une implémentation pour l'enregistrement des jeux de données Pascal VOC (voir [detectron2/detectron2/data/datasets/pascal_voc.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/data/datasets/pascal_voc.py) 

In [0]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator

import numpy as np
import os
import xml.etree.ElementTree as ET
from fvcore.common.file_io import PathManager
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode



# fmt: off
CLASS_NAMES= ['signature','Handwrit','intial']
# fmt: on

def load_voc_instances(dirname: str, split: str):
    """
    Load Pascal VOC detection annotations to Detectron2 format.
    Args:
        dirname: Contain "Annotations", "ImageSets", "JPEGImages"
        split (str): one of "train", "test", "val", "trainval"
    """
    with PathManager.open(os.path.join(dirname, "ImageSets", "Main", split + ".txt")) as f:
        fileids = np.loadtxt(f, dtype=np.str)

    # Needs to read many small annotation files. Makes sense at local
    annotation_dirname = PathManager.get_local_path(os.path.join(dirname, "Annotations/"))
    dicts = []
    for fileid in fileids:
        anno_file = os.path.join(annotation_dirname, fileid + ".xml")
        jpeg_file = os.path.join(dirname, "JPEGImages", fileid + ".jpg")

        with PathManager.open(anno_file) as f:
            tree = ET.parse(f)

        r = {
            "file_name": jpeg_file,
            "image_id": fileid,
            "height": int(tree.findall("./size/height")[0].text),
            "width": int(tree.findall("./size/width")[0].text),
        }
        instances = []

        for obj in tree.findall("object"):
            cls = obj.find("name").text
            # We include "difficult" samples in training.
            # Based on limited experiments, they don't hurt accuracy.
            # difficult = int(obj.find("difficult").text)
            # if difficult == 1:
            # continue
            bbox = obj.find("bndbox")
            bbox = [float(bbox.find(x).text) for x in ["xmin", "ymin", "xmax", "ymax"]]
            # Original annotations are integers in the range [1, W or H]
            # Assuming they mean 1-based pixel indices (inclusive),
            # a box with annotation (xmin=1, xmax=W) covers the whole image.
            # In coordinate space this is represented by (xmin=0, xmax=W)
            bbox[0] -= 1.0
            bbox[1] -= 1.0
            instances.append(
                {"category_id": CLASS_NAMES.index(cls), "bbox": bbox, "bbox_mode": BoxMode.XYXY_ABS}
            )
        r["annotations"] = instances
        dicts.append(r)
    return dicts

In [0]:
dirname ='/content/drive/My Drive/my-project/detectron/PascalVOC-export' 

#train
DatasetCatalog.register("data_train",lambda: load_voc_instances(dirname, "train"))
MetadataCatalog.get("data_train").set(thing_classes=CLASS_NAMES)

DatasetCatalog.register("data_val",lambda: load_voc_instances(dirname, "val"))
MetadataCatalog.get("data_val").set(thing_classes=CLASS_NAMES)


Metadata(name='data_val', thing_classes=['signature', 'Handwrit', 'intial'])

Vous pouvez basculer vers le jeu de données de test avec option --split test

Pour vérifier que le chargement des données est correct, visualisons les annotations d'échantillons sélectionnés au hasard dans l'ensemble d'apprentissage:


In [0]:
from google.colab.patches import cv2_imshow
import random
import cv2
from detectron2.utils.visualizer import Visualizer
samples=5
split ='val'

dataset_dicts = DatasetCatalog.get(dataset_name+split)
for d in random.sample(dataset_dicts, samples):
        img = cv2.imread(d["file_name"])
        visualizer = Visualizer(img[:, :, ::-1],
                                metadata=MetadataCatalog.get(dataset_name+split),
                                scale=0.5)
        vis = visualizer.draw_dataset_dict(d)
        cv2_imshow(vis.get_image()[:, :, ::-1])

ModuleNotFoundError: ignored

Nous sommes prêts à former notre modèle.

# Former un modèle personnalisé


Nous pouvons choisir la configuration du modèle et les poids du [Detectron2 Model Zoo](https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md)et créer DefaultTrainer- un entraîneur avec une logique d'entraînement par défaut.
Malheureusement, l'évaluateur pour l'ensemble de test n'est pas inclus par défaut. Mais nous pouvons facilement résoudre ce problème en écrivant notre propre fonction de train .















In [0]:
class CocoTrainer(DefaultTrainer):
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder is None:
      output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

Notre approche consistera à utiliser l'apprentissage par transfert où les poids de l'architecture de réseau existante sont ajustés pour prévoir les classes sur lesquelles le réseau original n'a pas été formé.

Maintenant, affinons un modèle R-CNN de FPN pré-entraîné de coco sur notre jeu de données.

Chargeons le fichier de configuration et les poids du modèle pré-formés:

In [0]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")

Précisez les ensembles de données que nous avons sauvegardés.

In [0]:
cfg.DATASETS.TRAIN = ("dataset_train","dataset_val")
cfg.DATASETS.TEST = ("dataset_val",) 
cfg.DATALOADER.NUM_WORKERS = 2 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6

In [0]:
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 4000
cfg.SOLVER.STEPS = (3000,)
cfg.SOLVER.GAMMA = 0.1

À l'exception des paramètres standard (taille des lots, nombre maximum d'itérations et taux d'apprentissage), nous avons quelques paramètres intéressants :


WARMUP_ITERS - le taux d'apprentissage commence à 0 et passe à celui prédéfini pour ce nombre d'itérations.

STEPS - les points de contrôle (nombre d'itérations) auxquels le taux d'apprentissage sera réduit de GAMMA


Enfin, nous spécifierons le nombre de classes et la période à laquelle nous évaluerons sur l'ensemble de test:


In [0]:
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.TEST.EVAL_PERIOD = 100

Il est temps de s'entraîner, en utilisant notre entraîneur personnalisé:

In [0]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("data_train",)
cfg.DATASETS.TEST = ("data_val",) 
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 20000
cfg.SOLVER.MAX_ITER = 30000
cfg.SOLVER.STEPS = (20000,30000)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.TEST.EVAL_PERIOD = 500

In [0]:
cfg.OUTPUT_DIR='/content/drive/My Drive/my-project/detectron/output'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[05/06 17:13:31 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f6e8b1.pkl: 243MB [00:24, 9.84MB/s]                           


[05/06 17:18:42 d2.engine.train_loop]: Starting training from iteration 0
[05/06 17:19:22 d2.utils.events]:  eta: 14:44:25  iter: 19  total_loss: 2.503  loss_cls: 1.306  loss_box_reg: 0.409  loss_rpn_cls: 0.656  loss_rpn_loc: 0.056  time: 1.9272  data_time: 1.4687  lr: 0.000002  max_mem: 5541M
[05/06 17:20:00 d2.utils.events]:  eta: 14:49:43  iter: 39  total_loss: 2.344  loss_cls: 1.257  loss_box_reg: 0.468  loss_rpn_cls: 0.620  loss_rpn_loc: 0.057  time: 1.9277  data_time: 1.4229  lr: 0.000003  max_mem: 5541M
[05/06 17:20:44 d2.utils.events]:  eta: 14:52:37  iter: 59  total_loss: 2.381  loss_cls: 1.179  loss_box_reg: 0.617  loss_rpn_cls: 0.506  loss_rpn_loc: 0.055  time: 2.0113  data_time: 1.6634  lr: 0.000004  max_mem: 5541M
[05/06 17:21:34 d2.utils.events]:  eta: 14:55:48  iter: 79  total_loss: 2.282  loss_cls: 1.114  loss_box_reg: 0.756  loss_rpn_cls: 0.340  loss_rpn_loc: 0.046  time: 2.1446  data_time: 2.0100  lr: 0.000005  max_mem: 5977M
[05/06 17:21:58 d2.utils.events]:  eta: 14

In [0]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
from google.colab.patches import cv2_imshow
import random
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
import os
cfg = get_cfg()
cfg.OUTPUT_DIR='/content/drive/My Drive/my-project/detectron/output'
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.DATASETS.TEST = ("data_val", )
CLASS_NAMES= ['signature','Handwrit','intial']
MetadataCatalog.get('data_val').set(thing_classes=CLASS_NAMES)
predictor = DefaultPredictor(cfg)



In [0]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = DatasetCatalog.get('data_val')
for d in random.sample(dataset_dicts, 10):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata= MetadataCatalog.get('data_val'), 
                   scale=0.8   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])


In [0]:
#file_path = '/content/drive/My Drive/my-project/workspace/training_faster_rcnn/images/LMD09443-SLA-5-_Z-1.jpg'
#im = cv2.imread(file_path)
#outputs = predictor(im)

In [0]:
# We can use `Visualizer` to draw the predictions on the image
#v = Visualizer(im[:, :, ::-1],MetadataCatalog.get('sig-hand_int_v'), scale=1.2)
#v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#cv2_imshow(v.get_image()[:, :, ::-1])

**Évaluation**

L'évaluation des modèles de détection d'objets est un peu différente par rapport à l'évaluation des modèles de classification ou de régression standard.

La métrique principale que vous devez connaître est IoU
Pour décider si une prédiction est correcte par rapport à un objet , IoU ou Jaccard  est utilisé. Il est défini comme l'intersection de la bbox prédite et de la bbox réelle divisée par leur union.

Une prédiction est considérée comme True :

* Vrai positif (TP): IoU> 0,5 et classe correcte détectée;
* Faux positif (FP): IoU <0,5 ou boîte englobante dupliquée;
* Faux négatif (FN): IoU> 0,5 mais mauvaise classe détectée.

Une fois nos métriques calculées (TP, FP, TN), nous pouvons calculer deux autres métriques cruciales, la précision et le rappel, et tracer la courbe Précision-Rappel (PR). La précision moyenne est alors l'aire sous la courbe interpolée . Enfin, le mAP est la moyenne des AP calculés pour toutes les classes

AP @ X (par exemple AP50) est simplement AP à un certain seuil IoU. Cela devrait vous donner une compréhension pratique de la façon dont les modèles de détection d'objets sont évalués.





Je vous suggère de lire les articles suivants, si vous voulez en savoir plus sur le sujet.

https://kharshit.github.io/blog/2019/09/20/evaluation-metrics-for-object-detection-and-segmentation

https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173

https://blog.zenggyu.com/en/post/2018-12-16/an-introduction-to-evaluation-metrics-for-object-detection/

https://github.com/rafaelpadilla/Object-Detection-Metrics


Vous pouvez observer toutes les métriques sur TensorBoard en cours d'exécution


# Faire des prédiction


Nous pouvons commencer à faire des prédictions en chargeant le modèle et en fixant un seuil minimum de 85% de certitude auquel nous considérerons les prédictions comme correctes:

Ensuite, nous sélectionnons au hasard plusieurs échantillons pour visualiser les résultats de la prédiction.

# Conclusion
Toutes nos félicitations! Vous connaissez maintenant les bases de Detectron2 pour la détection d'objets! Vous pourriez être surpris par les résultats, compte tenu du petit ensemble de données dont nous disposons. C'est la puissance des grands modèles pré-formés.

J'espère que cet article  vous aidera à former votre propre modèle.
